In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
train_data.shape

(4320, 27)

In [4]:
test_data.shape

(4320, 11)

In [5]:
train_data.head()

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [6]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9


In [7]:
train_data1=train_data.iloc[:,3:]

 - split train_data1 to 18 ,每个月20天,每天24小时,每小时18个特征
 - transform to 18,24*20

In [8]:
train_data1=train_data1.replace('NR',0)

In [9]:
month_data={}
for m in range(12):
    sample_data=np.empty((18,480),dtype=np.float)
    for d in range(20):
        sample_data[:,24*d:24*(d+1)]=train_data1.iloc[18*(m*20+d):18*(m*20+d+1),:]
    month_data[m]=sample_data

In [10]:
month_data[0].shape

(18, 480)

- from 9 hour data to predict No. 10 hour data
- split group per 9 hours data 
- 20 * 24 -9 = 471 groups per month
- 471,9 * 18 per month
- 12 * 471,9*18

In [11]:
x=np.empty((12*471,9*18))
y=np.empty((12*471,1))

In [12]:
for m in range(12):
    for d in range(471):
#         print(m,d)
#         print(month_data[m][:,9*d:9*(d+1)].shape)
        x[m*471+d:m*471+(d+1),:]=month_data[m][:,d:9+d].reshape(1,-1)
        y[m*471+d:m*471+(d+1),0]=month_data[m][9,9+d]

In [13]:
x1=pd.DataFrame(x)
x1=x1.apply(lambda x: (x-x.mean())/x.std())
x1.head()

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
0,-1.358133,-1.358719,-1.359102,-1.517992,-1.676543,-1.675830,-1.675028,-1.674061,-1.197040,0.775375,...,-1.598482,-1.138872,-0.764102,-1.044615,-1.324452,-1.043850,0.173173,0.266484,0.265656,-1.140720
1,-1.358133,-1.358719,-1.518065,-1.676883,-1.676543,-1.675830,-1.675028,-1.198262,-0.880015,0.775375,...,-1.411100,-0.763602,-1.045549,-1.325864,-1.043502,0.173616,0.266881,0.266484,-1.139531,-1.328212
2,-1.358133,-1.517759,-1.677028,-1.676883,-1.676543,-1.675830,-1.198999,-0.881062,-0.404478,0.775375,...,-0.661572,-1.045054,-1.326996,-1.044615,0.173945,0.267267,0.266881,-1.139134,-1.326889,-0.859484
3,-1.517230,-1.676800,-1.677028,-1.676883,-1.676543,-1.199572,-0.881646,-0.405263,-0.087454,0.775375,...,-0.380499,-1.326506,-1.045549,0.174130,0.267594,0.267267,-1.138731,-1.326549,-0.858493,-0.484501
4,-1.676327,-1.676800,-1.677028,-1.676883,-1.200071,-0.882066,-0.405616,-0.088063,-0.087454,0.775375,...,-0.193117,-1.045054,0.174053,0.267880,0.267594,-1.137501,-1.326146,-0.858010,-0.483777,0.265463


In [14]:
x2=torch.Tensor(x1.values)
y2=torch.Tensor(y)
w=torch.Tensor(np.random.normal(0,0.01,(x2.shape[1],1)))
b=torch.Tensor([1.])
for p in [w,b]:
    p.requires_grad_(requires_grad=True)


model by hand

In [15]:
def lin_reg(x,w,b):
    return torch.mm(x,w)+b
    

In [16]:
def squared_loss(y_hat,y):
    return (y_hat-y.view(y_hat.size()))**2

In [17]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr * p.grad/batch_size

In [18]:
def train_hand(num_epochs,lr,train_features,train_labels,batch_size):
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for i in range(num_epochs):
        for x,y in data_iter:
            l=squared_loss(lin_reg(x,w,b),y).mean()
            if w.grad is not None:
                w.grad.data.zero_()
                b.grad.data.zero_()
            l.backward()
            sgd([w,b],lr,batch_size)
        if (i+1) % 100 ==0:
            with torch.no_grad():
                print('epoch ',i+1,'train loss: ',squared_loss(lin_reg(train_features,w,b),train_labels).mean().item())
            

In [20]:
x2=torch.Tensor(x1.values)
y2=torch.Tensor(y)
w=torch.Tensor(np.random.normal(0,0.01,(x2.shape[1],1)))
b=torch.Tensor([1.])
for p in [w,b]:
    p.requires_grad_(requires_grad=True)

train_hand(10000,0.01,x2,y2,256)

epoch  100 train loss:  380.9410705566406
epoch  200 train loss:  279.9990539550781
epoch  300 train loss:  211.99411010742188
epoch  400 train loss:  164.7173614501953
epoch  500 train loss:  131.41778564453125
epoch  600 train loss:  107.8755874633789
epoch  700 train loss:  91.11438751220703
epoch  800 train loss:  79.11027526855469
epoch  900 train loss:  70.45252227783203
epoch  1000 train loss:  64.12333679199219
epoch  1100 train loss:  59.45973205566406
epoch  1200 train loss:  56.00725555419922
epoch  1300 train loss:  53.378814697265625
epoch  1400 train loss:  51.36820983886719
epoch  1500 train loss:  49.788265228271484
epoch  1600 train loss:  48.51777648925781
epoch  1700 train loss:  47.483455657958984
epoch  1800 train loss:  46.62392044067383
epoch  1900 train loss:  45.89391326904297
epoch  2000 train loss:  45.26203918457031
epoch  2100 train loss:  44.70806884765625
epoch  2200 train loss:  44.20965576171875
epoch  2300 train loss:  43.76535415649414
epoch  2400 tra

model by nn

In [21]:
def lr_net(feat_input,feat_output):
    return nn.Linear(feat_input,feat_output)

### test linear model

In [19]:
from sklearn.linear_model import Lasso,Ridge,LinearRegression
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import mean_squared_error

In [22]:
lasso=Lasso(alpha=0.005)
kf=KFold(n_splits=5,shuffle=True,random_state=7).split(x)
score = cross_val_score(lasso,x,y,scoring='neg_mean_squared_error',cv=kf)
print('score : ',np.mean(-score))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

score :  34.76870881410614


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


#### feats norm

In [23]:
month_data[0].shape

(18, 480)

In [26]:
all_train_data=pd.DataFrame()
month_list=[]
sample_data=month_data[0]
for m in range(1,12,1):
    sample_data=np.hstack((sample_data,month_data[m]))
all_train_data=pd.DataFrame(sample_data)

In [27]:
all_train_data.shape

(18, 5760)

In [59]:
all_train_data.T.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.527413,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.290152,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,-12.300000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [28]:
all_train_data.describe()

,0,1,2,3,4,5,6,7,8,9,...,5750,5751,5752,5753,5754,5755,5756,5757,5758,5759
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,...,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,16.839444,21.592222,15.356667,18.605000,21.858889,21.964444,20.765000,19.822222,30.152222,24.876111,...,12.212222,43.421111,32.013333,17.016667,26.269444,24.573889,25.279444,24.792778,22.931111,23.119444
std,22.018169,28.887701,21.934004,25.814773,34.266238,36.986731,34.861763,33.538186,58.974914,36.440285,...,18.531536,97.926387,75.904846,20.152310,37.060988,35.466682,37.516946,39.197999,35.628224,36.337805
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.025000,1.125000,0.700000,0.850000,1.725000,1.575000,1.825000,1.825000,1.035000,1.050000,...,1.300000,1.700000,1.000000,1.025000,1.350000,1.100000,1.550000,1.525000,1.800000,1.850000
50%,8.000000,5.600000,2.200000,4.400000,4.350000,3.150000,3.250000,2.100000,8.800000,16.000000,...,2.450000,4.900000,7.050000,10.050000,9.200000,7.800000,7.600000,7.650000,7.650000,7.650000
75%,23.750000,36.750000,23.750000,32.000000,24.750000,24.000000,21.000000,18.000000,20.500000,29.750000,...,15.500000,16.750000,27.000000,24.750000,39.000000,32.250000,24.750000,16.500000,21.500000,20.250000
max,77.000000,80.000000,67.000000,76.000000,110.000000,116.000000,106.000000,104.000000,232.000000,153.000000,...,59.000000,311.000000,327.000000,65.000000,121.000000,109.000000,108.000000,118.000000,108.000000,109.000000


In [31]:
all_train_data1=all_train_data.T
all_train_data1=all_train_data1.apply(lambda x : (x-x.mean()) / x.std())

In [32]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,2.921351e-15,-4.132630e-13,-1.393800e-14,5.078230e-15,3.474738e-15,-2.338465e-15,-3.438183e-15,-4.431640e-16,2.485088e-16,1.855094e-16,4.130921e-15,-9.199431e-16,5.026265e-16,3.205650e-13,8.574353e-17,5.245418e-16,5.851993e-16,1.040902e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.536816e+00,-1.518692e+01,-1.571093e+00,-1.341940e+00,-1.417839e+00,-1.636509e+00,-1.933149e+00,-1.705857e+00,-1.628736e+00,-1.345187e+00,-9.808389e-02,-3.310232e+00,-2.401358e+00,-1.121682e+01,-1.631707e+00,-1.673570e+00,-2.156207e+00,-1.611732e+00
25%,-7.197621e-01,-8.174309e-01,-4.276098e-01,-6.730107e-01,-4.538382e-01,-6.829821e-01,-6.529812e-01,-7.434694e-01,-7.134846e-01,-7.450388e-01,-9.808389e-02,-6.907360e-01,-6.401558e-01,-7.680033e-01,-8.807613e-01,-8.710141e-01,-7.482952e-01,-7.648192e-01
50%,7.513122e-02,-1.912605e-02,-1.494652e-01,-2.907653e-01,-2.347470e-01,-2.466224e-01,-2.966459e-01,-1.553437e-01,-1.795877e-01,-2.049049e-01,-9.808389e-02,1.325340e-01,-2.548928e-01,-2.180656e-01,-3.898786e-01,-3.905364e-01,-1.851306e-01,-2.002106e-01
75%,7.110459e-01,7.791788e-01,1.904894e-01,3.781641e-01,7.198057e-02,4.644824e-01,3.632342e-01,5.397139e-01,5.449866e-01,4.702623e-01,-9.808389e-02,8.061186e-01,3.505206e-01,3.318721e-01,5.918869e-01,6.390586e-01,5.657555e-01,5.526008e-01
max,2.141854e+00,2.375788e+00,2.219482e+01,1.108104e+01,1.264781e+01,5.797768e+00,7.753892e+00,1.064478e+01,5.273788e+00,5.436493e+00,3.607990e+01,1.928760e+00,1.058751e+01,6.381187e+00,2.127201e+00,2.128011e+00,5.071072e+00,4.975368e+00


In [34]:
all_train_data1.shape

(5760, 18)

In [33]:
x3=np.empty((12*471,18*9))
y3=np.empty((12*471,1))

In [36]:
for m in range(12):
    for g in range(471):
        x3[471*m+g:471*m+g+1,:]=all_train_data1.iloc[480*m:480*(m+1)].values[g:g+9,:].reshape(1,-1)
        y3[471*m+g:471*m+g+1,:]=all_train_data1.iloc[480*m:480*(m+1)].values[g+9,9]

In [46]:
x3[1,152:162]

array([-0.17958771,  0.5152735 , -0.09808389, -1.28947789,  6.73487971,
        0.88180982, -1.15231349, -0.05789803, -1.40532054, -1.32942776])

In [41]:
y3[0]

array([0.5152735])

In [45]:
x3[0]

array([-1.35567676,  0.77917878,  0.37591913,  0.56928682, -0.54147467,
        0.94932657,  0.6271863 , -0.85040131,  0.50685114,  0.27521402,
       -0.09808389,  0.28221947, -0.53008068,  0.88180982, -1.24631231,
       -1.30397195, -0.84215595, -1.1412249 , -1.35567676,  0.77917878,
        0.06686953,  0.09148008, -0.67292936, -0.14965355, -0.32304112,
       -0.10187773,  0.27803819,  1.05540732, -0.09808389, -0.39136512,
       -0.42000552,  0.88180982, -0.79720683, -0.83933422, -0.46671288,
       -0.76481918, -1.35567676,  0.77917878,  0.00505961, -0.09964261,
       -0.7167476 , -0.31126827, -0.46821475, -0.26227564,  0.20176721,
        0.87536271, -0.09808389, -0.46620785, -0.58511826,  0.88180982,
       -1.03742604, -1.64822627, -1.21759901, -1.04712347, -1.51465542,
        0.77917878, -0.05675031, -0.19520396, -0.19092882, -0.52136741,
       -0.48141236, -0.47613952, -0.29399419,  0.81534784, -0.09808389,
        0.05769127, -0.64015584,  0.33187212, -0.83898409, -1.09

In [43]:
all_train_data1.iloc[:10,:]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-1.355677,0.779179,0.375919,0.569287,-0.541475,0.949327,0.627186,-0.850401,0.506851,0.275214,-0.098084,0.282219,-0.530081,0.881810,-1.246312,-1.303972,-0.842156,-1.141225
1,-1.355677,0.779179,0.066870,0.091480,-0.672929,-0.149654,-0.323041,-0.101878,0.278038,1.055407,-0.098084,-0.391365,-0.420006,0.881810,-0.797207,-0.839334,-0.466713,-0.764819
2,-1.355677,0.779179,0.005060,-0.099643,-0.716748,-0.311268,-0.468215,-0.262276,0.201767,0.875363,-0.098084,-0.466208,-0.585118,0.881810,-1.037426,-1.648226,-1.217599,-1.047123
3,-1.514655,0.779179,-0.056750,-0.195204,-0.190929,-0.521367,-0.481412,-0.476140,-0.293994,0.815348,-0.098084,0.057691,-0.640156,0.331872,-0.838984,-1.092773,-1.593042,-1.329428
4,-1.673634,0.779179,-0.118560,-0.290765,-0.147111,-0.537529,-0.494610,-0.422674,-0.675349,0.575288,-0.098084,-0.091994,-0.475043,0.331872,-0.483877,-0.680935,-0.560574,-1.047123
5,-1.673634,0.779179,-0.273085,-0.768572,-0.278565,-1.022373,-0.916933,-0.208810,-1.171110,0.395244,-0.098084,-0.017151,-0.750231,-0.218066,-0.525655,-0.448616,0.190312,0.176195
6,-1.673634,0.779179,-0.056750,-0.386327,-0.103292,-0.521367,-0.455017,-0.422674,-1.476194,0.215199,-0.098084,0.057691,-0.695193,0.331872,-0.577876,-0.554216,0.190312,0.270297
7,-1.673634,0.779179,0.252299,-0.099643,0.028162,-0.375914,-0.309844,-0.529605,-1.552465,-0.084875,-0.098084,-0.017151,-0.640156,0.331872,-0.546543,-0.680935,-0.278991,0.270297
8,-1.196698,0.779179,1.210353,1.142655,1.956165,0.787712,1.287066,-0.583071,-1.209246,-0.144890,-0.098084,-0.541051,1.286159,1.431748,-0.337657,0.776338,-1.593042,-1.141225
9,-0.878741,0.779179,1.086733,0.855971,2.525802,1.757400,2.210899,-0.155344,-0.179588,0.515273,-0.098084,-1.289478,6.734880,0.881810,-1.152313,-0.057898,-1.405321,-1.329428


In [47]:
x3[5000,152:162]

array([-0.59907812, -0.62500902, -0.09808389, -0.01715146, -0.91534373,
       -0.21806557, -1.18364643,  1.62113377, -0.84215595, -0.95302204])

In [48]:
y3[4999]

array([-0.62500902])

In [52]:
lasso=Lasso(alpha=0.005)
kf=KFold(n_splits=5,shuffle=True,random_state=10).split(x3)
score = cross_val_score(lasso,x3,y3,scoring='neg_mean_squared_error',cv=kf)
print('score : ',np.mean(-score))

score :  0.12661058827144347


In [56]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,cross_val_score

In [57]:
data_idx=list(range(len(x3)))
np.random.shuffle(data_idx)
x_train=x3[data_idx[500:]]
y_train=y3[data_idx[500:]]
x_test=x3[data_idx[:500]]
y_test=y3[data_idx[:500]]

In [58]:
LR = linear_model.LinearRegression()
LR.fit(x_train, y_train)
y_pred = LR.predict(x_test)
print(mean_squared_error(y_pred, y_test))

0.14281836291740085


In [63]:
y[3700]

array([18.])

In [64]:
y3[3700]

array([-0.20490493])

In [ ]:
21.414236,16.662537

In [65]:
(18-21.414236)/16.662537

-0.20490493134388832

In [66]:
LR2 = linear_model.LinearRegression()
y_pred_train=LR.predict(x_train)
LR2.fit(y_pred_train, y[data_idx[500:]])
y_pred_test = LR2.predict(y_pred)
print(mean_squared_error(y_pred_test, y[data_idx[:500]]))

39.65211192109832
